In [2]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import matplotlib.cm as cm
import matplotlib.animation as animation
import netCDF4 as nc
import rioxarray
import xarray
import rasterio
from rasterio.plot import show
import os
import glob

In [3]:
output_folder = r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Yearly_outputs_new"

# Ensure the output folder exists; create it if necessary
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [4]:
# import xarray as xr
# import glob

nc_files = glob.glob(r'C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Yearly_inputs\*.nc')
target_var_names = []

def open_nc(input_nc, chunksize=None, layer=None):
    if chunksize is None:
        dts = xr.open_dataset(input_nc)
    else:
        dts = xr.open_dataset(input_nc,
                              chunks={'time': chunksize[0],
                                      'longitude': chunksize[1],
                                      'latitude': chunksize[2]})
    if type(layer) is int:  # select dataarray by index
        layer_name = list(dts.keys())[layer]
        return dts[layer_name]
    elif type(layer) is str:  # select dataarray by name
        return dts[layer]
    else:
        return dts

for i in nc_files:
    ncfile = open_nc(i, chunksize= None)
    # Remove 'spatial_ref' variable
    if 'spatial_ref' in ncfile.variables:
        ncfile = ncfile.drop_vars('spatial_ref')
    
    var_names = ncfile.variables.keys()
    
    for value in var_names:
        if value not in ['time', 'latitude', 'longitude']:
            target_var_names.append(value)
    
    print(i)
    print(target_var_names)

# If you want to accumulate all variable names across files, move this line outside the loop.
# print(target_var_names)


C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Yearly_inputs\Incomati_ETa_SSEBop_yearly.nc
['ETa']
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Yearly_inputs\Incomati_Incremental_ET_M_ETincr_yearly.nc
['ETa', 'Incremental_ET_M']
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Yearly_inputs\Incomati_P_CHIRPS_yearly.nc
['ETa', 'Incremental_ET_M', 'P']
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Yearly_inputs\Incomati_Rainfall_ET_M_ETrain_yearly.nc
['ETa', 'Incremental_ET_M', 'P', 'Rainfall_ET_M']
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Yearly_inputs\Incomati_SRO_M_SRO_yearly.nc
['ETa', 'Incremental_ET_M', 'P', 'Rainfall_ET_M', 'SRO_M']


In [5]:
# Loop through each netCDF file
def open_nc(input_nc, chunksize=None, layer=None):
    if chunksize is None:
        dts = xr.open_dataset(input_nc)
    else:
        dts = xr.open_dataset(input_nc,
                              chunks={'time': chunksize[0],
                                      'longitude': chunksize[1],
                                      'latitude': chunksize[2]})
    if type(layer) is int:  # select dataarray by index
        layer_name = list(dts.keys())[layer]
        return dts[layer_name]
    elif type(layer) is str:  # select dataarray by name
        return dts[layer]
    else:
        return dts

for nc_file in nc_files:
    # Open the netCDF file
    ncfile = xr.open_dataset(nc_file)
    ncfile = open_nc(nc_file, chunksize=None)

    # Remove 'spatial_ref' variable
    # if 'spatial_ref' in ncfile.variables:
    #     ncfile = ncfile.drop_vars('spatial_ref')

    # Get a list of all variable names
    var_names = ncfile.variables.keys()
    print(var_names)

    # Initialize the selected_variable variable as None
    selected_variable = None

    # Loop through the variable names and find the first match
    for var_name in var_names:
        if var_name in target_var_names:
            selected_variable = var_name
            break

    print(f"Selected variable for {nc_file}: {selected_variable}")

    # Load the nc file using xarray
    xds = xr.open_dataset(nc_file)
    
    # Use os.path.basename to get the file name including the extension
    file_name_with_extension = os.path.basename(nc_file)

    # Use os.path.splitext to split the file name into the name and extension
    file_name, file_extension = os.path.splitext(file_name_with_extension)

    # Now, 'file_name' contains the name of the file without the extension
    # Define the projection
    xds.rio.write_crs("epsg:4326", inplace=True)

    # Remove the grid_mapping attribute
    if 'grid_mapping' in xds[selected_variable].attrs:
        del xds[selected_variable].attrs['grid_mapping']

    output_tif = file_name + ".tif"
    full_file_path1 = os.path.join(output_folder, output_tif)

    print(output_tif)
    xds[selected_variable].rio.to_raster(full_file_path1, compress="LZW")
    print(f"Created TIFF file for {nc_file}: {full_file_path1}")

    # Open the TIFF file
    dataset = rasterio.open(full_file_path1)
    # Read and calculate the average
    band_count = dataset.count
#     print(band_count)
    sum_values = 0

    for band_index in range(1, band_count+1):
#         print(band_index)
        band = dataset.read(band_index)
        sum_values += band

    average = sum_values / (band_count)
    if np.any(average > 1000000):
        average = np.nan

    # Create a new raster file to save the average values
    avg_tif = file_name + "_avg.tif"
    
    full_file_path1 = os.path.join(output_folder, avg_tif)
#     avg_tif = glob.glob(os.path.join(output_folder, avg_tif))
    output_profile = dataset.profile
    output_profile.update(count=1, dtype=rasterio.float32)

    with rasterio.open(full_file_path1, 'w', **output_profile) as dst:
        dst.write(average.astype(rasterio.float32), 1)

    print(f"Created average TIFF file for {nc_file}: {avg_tif}")

    # Close the dataset
    dataset.close()
    


KeysView(Frozen({'time': <xarray.IndexVariable 'time' (time: 19)> Size: 152B
array(['2003-01-01T00:00:00.000000000', '2004-01-01T00:00:00.000000000',
       '2005-01-01T00:00:00.000000000', '2006-01-01T00:00:00.000000000',
       '2007-01-01T00:00:00.000000000', '2008-01-01T00:00:00.000000000',
       '2009-01-01T00:00:00.000000000', '2010-01-01T00:00:00.000000000',
       '2011-01-01T00:00:00.000000000', '2012-01-01T00:00:00.000000000',
       '2013-01-01T00:00:00.000000000', '2014-01-01T00:00:00.000000000',
       '2015-01-01T00:00:00.000000000', '2016-01-01T00:00:00.000000000',
       '2017-01-01T00:00:00.000000000', '2018-01-01T00:00:00.000000000',
       '2019-01-01T00:00:00.000000000', '2020-01-01T00:00:00.000000000',
       '2021-01-01T00:00:00.000000000'], dtype='datetime64[ns]'), 'latitude': <xarray.IndexVariable 'latitude' (latitude: 235)> Size: 940B
array([-24.103718, -24.11324 , -24.12276 , ..., -26.312805, -26.322327,
       -26.33185 ], dtype=float32)
Attributes:
    axis

In [ ]:
avg_tif_files = glob.glob(r'C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_yearly\*.tif')
x =0
for i in avg_tif_files:
    print(i, x)
    x += 1

In [ ]:
# Specify the paths to your TIFF files
P_Avg_tif = avg_tif_files[4]
ET_Avg_tif = avg_tif_files[0]

# Output TIFF file path for the difference
output_tif = output_folder+"\_Ghana_Water_yield_yearly_avg.tif"
print(output_tif)

# Open the TIFF files using rasterio
with rasterio.open(P_Avg_tif) as p_avg_ds, rasterio.open(ET_Avg_tif) as et_avg_ds:
    # Read the data from the TIFF files as numpy arrays
    p_avg_array = p_avg_ds.read(1)
    et_avg_array = et_avg_ds.read(1)

    # Calculate the difference between the two arrays
    difference_array = p_avg_array - et_avg_array

    # Copy the metadata from one of the input datasets
    profile = p_avg_ds.profile
    profile.update(count=1, dtype=rasterio.float32)

    # Create the output TIFF file
    with rasterio.open(output_tif, 'w', **profile) as output_ds:
        # Write the difference array to the output dataset
         output_ds.write(difference_array.astype(rasterio.float32), 1)

print(f"Difference saved to {output_tif}")


In [ ]:
import rasterio
import numpy as np

# Open the TIFF file for reading
with rasterio.open(r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\kenya\netcdfs\output_yearly\Kenya_SRO_yearly_avg.tif") as src:
    # Read the raster data as a NumPy array
    data = src.read(1)

    # Create a mask for values greater than 1000000
    # mask = data < 0
    mask1 = data > 1000000
    # Set values above 1000000 to NoData (typically represented as None)
    # data[mask] = src.nodata
    data[mask1] = src.nodata

    # Create an output TIFF file for writing
    with rasterio.open(r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\kenya\netcdfs\output_yearly\Kenya_SRO_M_SRO_yearly_avg_N.tif", 'w', **src.profile) as dst:
        # Write the modified data to the output file
        dst.write(data, 1)

print("Conversion complete.")